In [ ]:
import numpy
import pandas as pd
import os
import tarfile
from six.moves import urllib
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [51]:
dataframe = pd.read_csv("datasets/housing/housing.csv", sep=',', header=0)
dataset = dataframe.values

In [52]:
dataframe.head() 

,BIZPROP,ROOMS,AGE,HIGHWAYS,TAX,PTRATIO,LSTAT,VALUE
0,19.58,7.489,90.8,5,403,14.7,1.73,50.0
1,19.58,7.802,98.2,5,403,14.7,1.92,50.0
2,19.58,8.375,93.9,5,403,14.7,3.32,50.0
3,19.58,7.929,96.2,5,403,14.7,3.70,50.0
4,2.46,7.831,53.6,3,193,17.8,4.45,50.0


In [ ]:
#split the dataset into features and target values 
features = dataset[:,0:7]
target = dataset[:,7]

In [ ]:
# fix random seed for reproducibility 
seed = 9 
numpy.random.seed(seed)

In [ ]:
def simple_shallow_seq_net():
   # create a sequential ANN 
    model = Sequential() 
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid')) 
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [33]:
#divides the code into 10 random subsets for training and testing
seed = 9 
kfold = KFold(n_splits=10, random_state=seed)

/Users/parmin/anaconda3/envs/ai-projects/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [35]:
estimator = KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0)
# epochs runs the network 100 times 
#batch_size ensure that every learning cycle 50 training samples used at a time

In [36]:
results = cross_val_score(estimator, features, target, cv=kfold) 
print("simple_shallow_seq_model:(%.2f) MSE" % (results.std())) 

simple_shallow_seq_model:(166.93) MSE


In [37]:
# saves the model use for later
estimator.fit(features, target)
estimator.model.save('simple_shallow_seq_net.h5')

In [38]:
#lower the MSE to make the neuron more efficient 
estimators = [] 
estimators.append(('standardize', StandardScaler())) 
estimators.append(('estimator', KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0))) 
pipeline = Pipeline(estimators)

In [39]:
#prints out the MSE to see if the value has decreased
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(69.83) MSE


In [40]:
#saves the value for later use
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('standardised_shallow_seq_net.h5')

In [41]:
def deep_seq_net(): 
    # create a deep sequential model 
    model = Sequential() 
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid')) 
    model.add(Dense(7,activation='tanh')) 
    model.add(Dense(7,activation='sigmoid')) 
    model.add(Dense(7,activation='tanh')) 
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [48]:

# Standardise the data and fit the neural net model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_seq_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)


In [56]:
#cross validate the fir model
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(57.47) MSE


In [50]:
#save this model for later
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('deep_seq_net.h5')

In [53]:
def deep_and_wide_net(): 
    # create a sequential model 
    model = Sequential() 
    model.add(Dense(21, input_dim=7, kernel_initializer='normal', activation='relu')) 
    model.add(Dense(21,activation='relu')) 
    model.add(Dense(21,activation='relu')) 
    model.add(Dense(21,activation='sigmoid')) 
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [54]:
estimators = [] 
estimators.append(('standardize', StandardScaler())) 
estimators.append(('estimator', KerasRegressor(build_fn=deep_and_wide_net, epochs=100, batch_size=50, verbose=0))) 
pipeline = Pipeline(estimators)

In [57]:
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("deep_and_wide_model:(%.2f) MSE" % (results.std()))

deep_and_wide_model:(52.63) MSE


In [58]:
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('deep_and_wide_net.h5')